# Extracting player information

- For a comparision, the 1st match has an id='335982' and it's scoreboard can be found in the [cricinfo page.](https://www.espncricinfo.com/series/8048/scorecard/335982/royal-challengers-bangalore-vs-kolkata-knight-riders-1st-match-indian-premier-league-2007-08)

- The ball-by-ball data can be downloaded from (https://cricsheet.org/downloads/).



In [1]:
import numpy as np
import pandas as pd
import pickle

from utils import get_match_list
from print_scorecard import print_scorecard

data_dir='../datasets/ipl/'

In [2]:
import psycopg2

# load the postgress credentials
import sys
sys.path.insert(0, '/Users/gshyam/creds/')
from load_credentials import load_credentials
fil='/Users/gshyam/creds/postgress.dat'
my_info = load_credentials(fil)

con = psycopg2.connect(
    host="127.0.0.1",
    database="myfirstdb",
    user = my_info["user"],
    password = my_info["password"]
)
# cursor
c = con.cursor()

## Generalize the submission

In [8]:


col_names = ['batsman', 'zeros', 'ones', 'twos', 'threes', 'fours',
             'fives', 'sixes', 'runs', 'balls_faced', 'not_out', 'team', 'against',
             'tied', 'win', 'toss', 'team_total', 'season', 'date', 'match_id']
col_types = ['text', 'int', 'int', 'int', 'int', 'int',
             'int', 'int', 'int', 'int', 'boolean', 'text', 'text',
             'boolean', 'boolean', 'boolean', 'int', 'int', 'text', 'int']

#col_names = ['batsman', 'runs', 'balls_faced', 'not_out', 'team', 'match_id']
#col_types = ['text', 'int', 'int',  'boolean', 'text', 'int']

def create_table_full(table_name, col_names=col_names, col_types=col_types):
    cmd = "CREATE TABLE  "+table_name+" ( "
    
    for i in range(len(col_names)):
        cmd = cmd + col_names[i] +" " +col_types[i]+", "
    cmd = cmd[:-2] + """ )"""  # :-2 to get rid of last comma
    #cmd = cmd + """ PRIMARY KEY (match_id) )"""
    c.execute(cmd)
    con.commit()

#table_name = 'gri5'
#create_table_full(table_name)

In [9]:
def push_to_database_(match_id, table_name):
    df_bat, df_bowl = print_scorecard(str(match_id)+'.yaml', data_dir=data_dir)
    batters = df_bat.batsman.unique()
    for batter in batters:
        data = df_bat[df_bat['batsman']==batter].values[0]        

        cmd = "INSERT INTO "+ table_name+" VALUES ("
        for i in range(len(col_names)):
            if col_types[i]=='text':
                cmd = cmd + "'"+data[i]+"', "
            else:
                cmd = cmd + str(data[i]) +", "
        cmd = cmd[:-2]+")" # -2 to get rid of comma from the last entry
        c.execute(cmd)
        con.commit()

match_id = 335982
push_to_database_(match_id, table_name)

In [8]:
def fetch_all_data(table_name):
    c.execute("SELECT * FROM "+table_name)
    return pd.DataFrame(c.fetchall())

def fetch_all_data2(table_name):
    
    #"""SELECT table_name 
    #FROM information_schema.tables 
    #WHERE table_name LIKE 'main_%'
    #"""
    cmd = "SELECT * FROM "+table_name
    table_list = pd.read_sql(cmd, con)
    
    

cmd = "SELECT * FROM "+table_name
dff = pd.read_sql(cmd, con)

dff
    
#fetch_all_data(table_name)

,batsman,zeros,ones,twos,threes,fours,fives,sixes,runs,balls_faced,not_out,team,against,tied,win,toss,team_total,season,date,match_id
0,SC Ganguly,8,2,0,0,2,0,0,10,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
1,BB McCullum,23,22,9,0,10,0,13,158,73,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
2,RT Ponting,9,8,1,0,1,0,1,20,20,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
3,DJ Hussey,4,6,1,0,1,0,0,12,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
4,Mohammad Hafeez,1,1,0,0,1,0,0,5,3,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,YK Pathan,17,12,4,0,3,0,4,56,39,False,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040
880,M Kaif,3,4,1,0,0,0,1,12,9,False,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040
881,RA Jadeja,1,0,0,0,0,0,0,0,1,False,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040
882,SK Warne,3,5,0,0,1,0,0,9,9,True,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040


In [16]:
def convert_data_to_df(table_name):
    c.execute("SELECT * FROM "+table_name)
    rows = c.fetchall()
    df = pd.DataFrame(rows)
    display (df.head(5))
convert_data_to_df(table_name)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,SC Ganguly,8,2,0,0,2,0,0,10,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
1,BB McCullum,23,22,9,0,10,0,13,158,73,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
2,RT Ponting,9,8,1,0,1,0,1,20,20,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
3,DJ Hussey,4,6,1,0,1,0,0,12,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
4,Mohammad Hafeez,1,1,0,0,1,0,0,5,3,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982


### Real Table
**which should look like actual table**

In [18]:
table_name = 'realtable'
create_table_full(table_name)

In [21]:
def push_all(table_name):
    years=range(2008, 2009, 1)
    for year in years:
        df_match=get_match_list(year, data_dir=data_dir)
        match_ids = df_match['match_id']
        
        for match_id in match_ids:
            df_bat, df_bowl = print_scorecard(str(match_id)+'.yaml', data_dir=data_dir)
            batters = df_bat.batsman.unique()
            for batter in batters:
                data = df_bat[df_bat['batsman']==batter].values[0]

                cmd = "INSERT INTO "+ table_name+" VALUES ("
                for i in range(len(col_names)):
                    if col_types[i]=='text':
                        cmd = cmd + "'"+data[i]+"', "
                    else:
                        cmd = cmd + str(data[i]) +", "
                cmd = cmd[:-2]+")" # -2 to get rid of comma from the last entry
                #print (cmd)
                c.execute(cmd)
                con.commit()

push_all(table_name)

INSERT INTO realtable VALUES ('SC Ganguly', 8, 2, 0, 0, 2, 0, 0, 10, 12, False, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO realtable VALUES ('BB McCullum', 23, 22, 9, 0, 10, 0, 13, 158, 73, True, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO realtable VALUES ('RT Ponting', 9, 8, 1, 0, 1, 0, 1, 20, 20, False, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO realtable VALUES ('DJ Hussey', 4, 6, 1, 0, 1, 0, 0, 12, 12, False, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO realtable VALUES ('Mohammad Hafeez', 1, 1, 0, 0, 1, 0, 0, 5, 3, True, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO realtable VALUES ('R Dravid', 1, 2, 0

INSERT INTO realtable VALUES ('L Ronchi', 8, 0, 0, 0, 2, 0, 0, 8, 10, False, 'Mumbai Indians', 'Royal Challengers Bangalore', False, False, True, 165, 2008, '2008-04-20', 335985)
INSERT INTO realtable VALUES ('ST Jayasuriya', 8, 3, 0, 0, 5, 0, 1, 29, 16, False, 'Mumbai Indians', 'Royal Challengers Bangalore', False, False, True, 165, 2008, '2008-04-20', 335985)
INSERT INTO realtable VALUES ('DJ Thornely', 6, 0, 0, 0, 0, 0, 0, 0, 5, False, 'Mumbai Indians', 'Royal Challengers Bangalore', False, False, True, 165, 2008, '2008-04-20', 335985)
INSERT INTO realtable VALUES ('RV Uthappa', 13, 16, 3, 0, 5, 0, 1, 48, 38, False, 'Mumbai Indians', 'Royal Challengers Bangalore', False, False, True, 165, 2008, '2008-04-20', 335985)
INSERT INTO realtable VALUES ('PR Shah', 11, 9, 1, 0, 2, 0, 0, 19, 23, False, 'Mumbai Indians', 'Royal Challengers Bangalore', False, False, True, 165, 2008, '2008-04-20', 335985)
INSERT INTO realtable VALUES ('AM Nayar', 4, 8, 1, 0, 1, 0, 1, 20, 14, False, 'Mumbai India

INSERT INTO realtable VALUES ('AC Gilchrist', 5, 1, 0, 0, 3, 0, 0, 13, 9, False, 'Deccan Chargers', 'Rajasthan Royals', False, False, False, 214, 2008, '2008-04-24', 335990)
INSERT INTO realtable VALUES ('VVS Laxman', 9, 4, 0, 0, 3, 0, 0, 16, 16, False, 'Deccan Chargers', 'Rajasthan Royals', False, False, False, 214, 2008, '2008-04-24', 335990)
INSERT INTO realtable VALUES ('Shahid Afridi', 2, 0, 0, 0, 1, 0, 1, 10, 4, False, 'Deccan Chargers', 'Rajasthan Royals', False, False, False, 214, 2008, '2008-04-24', 335990)
INSERT INTO realtable VALUES ('A Symonds', 13, 22, 3, 1, 11, 0, 7, 117, 53, True, 'Deccan Chargers', 'Rajasthan Royals', False, False, False, 214, 2008, '2008-04-24', 335990)
INSERT INTO realtable VALUES ('RG Sharma', 10, 14, 2, 0, 3, 0, 1, 36, 30, False, 'Deccan Chargers', 'Rajasthan Royals', False, False, False, 214, 2008, '2008-04-24', 335990)
INSERT INTO realtable VALUES ('AS Yadav', 4, 2, 1, 0, 3, 0, 0, 16, 9, False, 'Deccan Chargers', 'Rajasthan Royals', False, False,

INSERT INTO realtable VALUES ('S Chanderpaul', 7, 2, 0, 0, 0, 0, 0, 2, 9, False, 'Royal Challengers Bangalore', 'Rajasthan Royals', False, False, False, 135, 2008, '2008-04-26', 335992)
INSERT INTO realtable VALUES ('R Dravid', 1, 0, 0, 0, 0, 0, 0, 0, 1, False, 'Royal Challengers Bangalore', 'Rajasthan Royals', False, False, False, 135, 2008, '2008-04-26', 335992)
INSERT INTO realtable VALUES ('LRPL Taylor', 10, 2, 0, 0, 6, 0, 3, 44, 20, False, 'Royal Challengers Bangalore', 'Rajasthan Royals', False, False, False, 135, 2008, '2008-04-26', 335992)
INSERT INTO realtable VALUES ('JH Kallis', 5, 3, 0, 0, 2, 0, 0, 11, 10, False, 'Royal Challengers Bangalore', 'Rajasthan Royals', False, False, False, 135, 2008, '2008-04-26', 335992)
INSERT INTO realtable VALUES ('V Kohli', 9, 5, 1, 0, 0, 0, 1, 13, 14, False, 'Royal Challengers Bangalore', 'Rajasthan Royals', False, False, False, 135, 2008, '2008-04-26', 335992)
INSERT INTO realtable VALUES ('MV Boucher', 6, 4, 0, 0, 0, 0, 0, 4, 10, False, '

INSERT INTO realtable VALUES ('BB McCullum', 1, 1, 0, 0, 0, 0, 0, 1, 2, False, 'Kolkata Knight Riders', 'Mumbai Indians', False, False, True, 137, 2008, '2008-04-29', 335997)
INSERT INTO realtable VALUES ('SC Ganguly', 1, 0, 0, 0, 1, 0, 0, 4, 2, False, 'Kolkata Knight Riders', 'Mumbai Indians', False, False, True, 137, 2008, '2008-04-29', 335997)
INSERT INTO realtable VALUES ('RT Ponting', 15, 15, 0, 0, 1, 0, 0, 19, 31, False, 'Kolkata Knight Riders', 'Mumbai Indians', False, False, True, 137, 2008, '2008-04-29', 335997)
INSERT INTO realtable VALUES ('DB Das', 12, 3, 0, 0, 5, 0, 1, 29, 20, False, 'Kolkata Knight Riders', 'Mumbai Indians', False, False, True, 137, 2008, '2008-04-29', 335997)
INSERT INTO realtable VALUES ('DJ Hussey', 12, 5, 0, 0, 3, 0, 0, 17, 20, False, 'Kolkata Knight Riders', 'Mumbai Indians', False, False, True, 137, 2008, '2008-04-29', 335997)
INSERT INTO realtable VALUES ('Mohammad Hafeez', 4, 2, 0, 0, 0, 0, 0, 2, 6, False, 'Kolkata Knight Riders', 'Mumbai Indians'

...

INSERT INTO realtable VALUES ('Salman Butt', 22, 19, 4, 0, 10, 0, 1, 73, 54, False, 'Kolkata Knight Riders', 'Chennai Super Kings', False, False, True, 149, 2008, '2008-05-18', 336025)
INSERT INTO realtable VALUES ('Mohammad Hafeez', 3, 2, 0, 0, 1, 0, 0, 6, 6, False, 'Kolkata Knight Riders', 'Chennai Super Kings', False, False, True, 149, 2008, '2008-05-18', 336025)
INSERT INTO realtable VALUES ('SC Ganguly', 7, 0, 1, 0, 0, 0, 0, 2, 8, False, 'Kolkata Knight Riders', 'Chennai Super Kings', False, False, True, 149, 2008, '2008-05-18', 336025)
INSERT INTO realtable VALUES ('DB Das', 18, 11, 0, 0, 1, 0, 2, 27, 32, False, 'Kolkata Knight Riders', 'Chennai Super Kings', False, False, True, 149, 2008, '2008-05-18', 336025)
INSERT INTO realtable VALUES ('DJ Hussey', 1, 0, 0, 0, 0, 0, 0, 0, 1, False, 'Kolkata Knight Riders', 'Chennai Super Kings', False, False, True, 149, 2008, '2008-05-18', 336025)
INSERT INTO realtable VALUES ('LR Shukla', 6, 3, 1, 0, 2, 0, 0, 13, 11, True, 'Kolkata Knight R

INSERT INTO realtable VALUES ('JH Kallis', 3, 1, 0, 0, 1, 0, 0, 5, 5, False, 'Royal Challengers Bangalore', 'Chennai Super Kings', False, True, True, 126, 2008, '2008-05-21', 336029)
INSERT INTO realtable VALUES ('SP Goswami', 13, 4, 1, 0, 1, 0, 0, 10, 19, False, 'Royal Challengers Bangalore', 'Chennai Super Kings', False, True, True, 126, 2008, '2008-05-21', 336029)
INSERT INTO realtable VALUES ('V Kohli', 10, 6, 0, 0, 1, 0, 0, 10, 16, False, 'Royal Challengers Bangalore', 'Chennai Super Kings', False, True, True, 126, 2008, '2008-05-21', 336029)
INSERT INTO realtable VALUES ('R Dravid', 12, 17, 5, 0, 5, 0, 0, 47, 39, False, 'Royal Challengers Bangalore', 'Chennai Super Kings', False, True, True, 126, 2008, '2008-05-21', 336029)
INSERT INTO realtable VALUES ('Misbah-ul-Haq', 3, 2, 1, 0, 0, 0, 0, 4, 6, False, 'Royal Challengers Bangalore', 'Chennai Super Kings', False, True, True, 126, 2008, '2008-05-21', 336029)
INSERT INTO realtable VALUES ('MV Boucher', 7, 5, 0, 0, 3, 0, 0, 17, 15, 

INSERT INTO realtable VALUES ('GC Smith', 13, 21, 5, 0, 9, 0, 4, 91, 51, False, 'Rajasthan Royals', 'Chennai Super Kings', False, True, True, 211, 2008, '2008-05-24', 336033)
INSERT INTO realtable VALUES ('SA Asnodkar', 11, 9, 0, 0, 5, 0, 2, 41, 27, False, 'Rajasthan Royals', 'Chennai Super Kings', False, True, True, 211, 2008, '2008-05-24', 336033)
INSERT INTO realtable VALUES ('Kamran Akmal', 6, 11, 4, 0, 4, 0, 3, 53, 28, True, 'Rajasthan Royals', 'Chennai Super Kings', False, True, True, 211, 2008, '2008-05-24', 336033)
INSERT INTO realtable VALUES ('YK Pathan', 2, 2, 0, 0, 1, 0, 0, 6, 5, False, 'Rajasthan Royals', 'Chennai Super Kings', False, True, True, 211, 2008, '2008-05-24', 336033)
INSERT INTO realtable VALUES ('M Kaif', 2, 2, 2, 0, 1, 0, 0, 10, 7, False, 'Rajasthan Royals', 'Chennai Super Kings', False, True, True, 211, 2008, '2008-05-24', 336033)
INSERT INTO realtable VALUES ('T Kohli', 0, 3, 0, 0, 0, 0, 0, 3, 3, False, 'Rajasthan Royals', 'Chennai Super Kings', False, True

INSERT INTO realtable VALUES ('ST Jayasuriya', 19, 11, 4, 1, 4, 0, 0, 38, 37, False, 'Mumbai Indians', 'Rajasthan Royals', False, False, False, 145, 2008, '2008-05-26', 336036)
INSERT INTO realtable VALUES ('SR Tendulkar', 16, 12, 5, 0, 2, 0, 0, 30, 34, False, 'Mumbai Indians', 'Rajasthan Royals', False, False, False, 145, 2008, '2008-05-26', 336036)
INSERT INTO realtable VALUES ('AM Nayar', 5, 9, 2, 0, 3, 0, 0, 25, 18, False, 'Mumbai Indians', 'Rajasthan Royals', False, False, False, 145, 2008, '2008-05-26', 336036)
INSERT INTO realtable VALUES ('RV Uthappa', 7, 1, 1, 0, 0, 0, 0, 3, 8, False, 'Mumbai Indians', 'Rajasthan Royals', False, False, False, 145, 2008, '2008-05-26', 336036)
INSERT INTO realtable VALUES ('DR Smith', 4, 2, 1, 0, 1, 0, 0, 8, 7, False, 'Mumbai Indians', 'Rajasthan Royals', False, False, False, 145, 2008, '2008-05-26', 336036)
INSERT INTO realtable VALUES ('SM Pollock', 1, 0, 0, 0, 0, 0, 0, 0, 1, False, 'Mumbai Indians', 'Rajasthan Royals', False, False, False, 14

INSERT INTO realtable VALUES ('GC Smith', 14, 1, 2, 0, 5, 0, 0, 25, 21, False, 'Rajasthan Royals', 'Delhi Daredevils', False, True, False, 192, 2008, '2008-05-30', 336038)
INSERT INTO realtable VALUES ('SA Asnodkar', 7, 5, 2, 0, 6, 0, 1, 39, 21, False, 'Rajasthan Royals', 'Delhi Daredevils', False, True, False, 192, 2008, '2008-05-30', 336038)
INSERT INTO realtable VALUES ('Sohail Tanvir', 3, 1, 0, 0, 1, 0, 0, 5, 5, False, 'Rajasthan Royals', 'Delhi Daredevils', False, True, False, 192, 2008, '2008-05-30', 336038)
INSERT INTO realtable VALUES ('SR Watson', 9, 10, 4, 0, 4, 0, 3, 52, 29, False, 'Rajasthan Royals', 'Delhi Daredevils', False, True, False, 192, 2008, '2008-05-30', 336038)
INSERT INTO realtable VALUES ('M Kaif', 5, 6, 0, 0, 0, 0, 1, 12, 11, False, 'Rajasthan Royals', 'Delhi Daredevils', False, True, False, 192, 2008, '2008-05-30', 336038)
INSERT INTO realtable VALUES ('YK Pathan', 5, 9, 0, 0, 3, 0, 4, 45, 21, False, 'Rajasthan Royals', 'Delhi Daredevils', False, True, False,

In [7]:
table_name = 'realtable'
df = fetch_all_data(table_name)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,SC Ganguly,8,2,0,0,2,0,0,10,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
1,BB McCullum,23,22,9,0,10,0,13,158,73,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
2,RT Ponting,9,8,1,0,1,0,1,20,20,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
3,DJ Hussey,4,6,1,0,1,0,0,12,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
4,Mohammad Hafeez,1,1,0,0,1,0,0,5,3,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879,YK Pathan,17,12,4,0,3,0,4,56,39,False,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040
880,M Kaif,3,4,1,0,0,0,1,12,9,False,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040
881,RA Jadeja,1,0,0,0,0,0,0,0,1,False,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040
882,SK Warne,3,5,0,0,1,0,0,9,9,True,Rajasthan Royals,Chennai Super Kings,False,True,True,164,2008,2008-06-01,336040


In [29]:
def fetch_condition(table_name):
    cmd = "SELECT * FROM "+table_name+" WHERE batsman='MS Dhoni' "
    print (cmd)
    c.execute(cmd)
    #rows = c.fetchall()
    #df = pd.DataFrame(rows)
    #display (df.head(5))
fetch_condition(table_name)

SELECT * FROM realtable WHERE batsman='MS Dhoni' 


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
